In [31]:
import os
import time
import datetime
import moxing as mox
import argparse
import glob
import numpy as np
import mindspore.nn as nn
from mindspore import Tensor, context
from mindspore.nn.optim.momentum import Momentum
from mindspore.train.model import Model
from mindspore.train.serialization import load_checkpoint, load_param_into_net
from mindspore.ops import operations as P
from mindspore.ops import functional as F
from mindspore.common import dtype as mstype
import mindspore.dataset.vision.py_transforms as vision

In [32]:
mox.file.copy_parallel(src_url="s3://nlp.final/san/data/",dst_url="./data")

INFO:root:Listing OBS: 1000
INFO:root:Listing OBS: 2000
INFO:root:Listing OBS: 3000
INFO:root:Listing OBS: 4000
INFO:root:Listing OBS: 5000
INFO:root:Listing OBS: 6000
INFO:root:Listing OBS: 7000
INFO:root:Listing OBS: 8000
INFO:root:Listing OBS: 9000
INFO:root:Listing OBS: 10000
INFO:root:Listing OBS: 11000
INFO:root:Listing OBS: 12000
INFO:root:Listing OBS: 13000
INFO:root:Listing OBS: 14000
INFO:root:Listing OBS: 15000
INFO:root:Listing OBS: 16000
INFO:root:Listing OBS: 17000
INFO:root:Listing OBS: 18000
INFO:root:Listing OBS: 19000
INFO:root:Listing OBS: 20000
INFO:root:Listing OBS: 21000
INFO:root:Listing OBS: 22000
INFO:root:Listing OBS: 23000
INFO:root:Listing OBS: 24000
INFO:root:Listing OBS: 25000
INFO:root:Listing OBS: 26000
INFO:root:Listing OBS: 27000
INFO:root:Listing OBS: 28000
INFO:root:Listing OBS: 29000
INFO:root:Listing OBS: 30000
INFO:root:Listing OBS: 31000
INFO:root:Listing OBS: 32000
INFO:root:Listing OBS: 33000
INFO:root:Listing OBS: 34000
INFO:root:Listing OBS: 

In [3]:
mox.file.copy_parallel(src_url="s3://nlp.final/san/code/image_model/src",dst_url="./src/")

In [4]:
from PIL import Image, ImageFile

from src.utils.logging import get_logger
from src.dataset import classification_dataset
from easydict import EasyDict as edict
from src.config import imagenet_cfg as vgg_cfg
# from src.config import cifar_cfg as vgg_cfg

In [5]:
from src.vgg import vgg19
from src.vgg import vgg16

In [6]:
image_path = {
    "train":"./data/images/train",
    "test":"./data/images/test",
    "val":"./data/images/val"
}

annotation_path = {
    "train":"./data/annotation/train.json",
    "test":"./data/annotation/test.json",
    "val":"./data/annotation/val.json"
}

In [7]:
model_cfg= edict({
    "annotation":annotation_path,
    "image":image_path,
    "cnn_ckpt_19":"./data/vgg19_ascend_v111_imagenet2012_research_cv_bs64_acc74.ckpt",
    "cnn_ckpt_16":"./data/vgg16_ascend_v120_imagenet2012_official_cv_bs32_acc73.ckpt",
    "log_path":"./outputs",

    "device_target": 'Ascend',
    "per_batch_size": 32,
    "graph_ckpt":1,
    "rank": 0,
    "group_size":1
})

In [8]:
context.set_context(mode=context.GRAPH_MODE, enable_auto_mixed_precision=True,
                        device_target=model_cfg.device_target, save_graphs=False)
if os.getenv('DEVICE_ID', "not_set").isdigit() and model_cfg.device_target == "Ascend":
    context.set_context(device_id=int(os.getenv('DEVICE_ID')))
model_cfg.outputs_dir = os.path.join(model_cfg.log_path,
                                    datetime.datetime.now().strftime('%Y-%m-%d_time_%H_%M_%S'))

model_cfg.logger = get_logger(model_cfg.outputs_dir, model_cfg.rank)

In [9]:
model_cfg.logger.important_info('start create vgg')
network = vgg16(vgg_cfg.num_classes, vgg_cfg, phase="test",include_top = False)
network.add_flags_recursive(fp16=False)
network.set_train(False)

2021-07-13 03:15:06,155:INFO:
**********************************************************************
**********************************************************************
**
**
**        start create vgg
**
**
**********************************************************************
**********************************************************************

init network


Vgg<
  (layers): SequentialCell<
    (0): Conv2d<input_channels=3, output_channels=64, kernel_size=(3, 3),stride=(1, 1),  pad_mode=pad, padding=1, dilation=(1, 1), group=1, has_bias=Falseweight_init=ones, bias_init=zeros, format=NCHW>
    (1): ReLU<>
    (2): Conv2d<input_channels=64, output_channels=64, kernel_size=(3, 3),stride=(1, 1),  pad_mode=pad, padding=1, dilation=(1, 1), group=1, has_bias=Falseweight_init=ones, bias_init=zeros, format=NCHW>
    (3): ReLU<>
    (4): MaxPool2d<kernel_size=2, stride=2, pad_mode=VALID>
    (5): Conv2d<input_channels=64, output_channels=128, kernel_size=(3, 3),stride=(1, 1),  pad_mode=pad, padding=1, dilation=(1, 1), group=1, has_bias=Falseweight_init=ones, bias_init=zeros, format=NCHW>
    (6): ReLU<>
    (7): Conv2d<input_channels=128, output_channels=128, kernel_size=(3, 3),stride=(1, 1),  pad_mode=pad, padding=1, dilation=(1, 1), group=1, has_bias=Falseweight_init=ones, bias_init=zeros, format=NCHW>
    (8): ReLU<>
    (9): MaxPool2d<kernel_siz

In [10]:
model_cfg.logger.important_info('start load checkpoint')
param_dict = load_checkpoint(model_cfg.cnn_ckpt_16)

2021-07-13 03:15:23,903:INFO:
**********************************************************************
**********************************************************************
**
**
**        start load checkpoint
**
**
**********************************************************************
**********************************************************************



In [11]:
param_dict

{'classifier.0.bias': Parameter (name=classifier.0.bias, shape=(4096,), dtype=Float32, requires_grad=True),
 'classifier.3.bias': Parameter (name=classifier.3.bias, shape=(4096,), dtype=Float32, requires_grad=True),
 'classifier.6.bias': Parameter (name=classifier.6.bias, shape=(1000,), dtype=Float32, requires_grad=True),
 'layers.0.weight': Parameter (name=layers.0.weight, shape=(64, 3, 3, 3), dtype=Float32, requires_grad=True),
 'layers.2.weight': Parameter (name=layers.2.weight, shape=(64, 64, 3, 3), dtype=Float32, requires_grad=True),
 'layers.5.weight': Parameter (name=layers.5.weight, shape=(128, 64, 3, 3), dtype=Float32, requires_grad=True),
 'layers.7.weight': Parameter (name=layers.7.weight, shape=(128, 128, 3, 3), dtype=Float32, requires_grad=True),
 'layers.10.weight': Parameter (name=layers.10.weight, shape=(256, 128, 3, 3), dtype=Float32, requires_grad=True),
 'layers.12.weight': Parameter (name=layers.12.weight, shape=(256, 256, 3, 3), dtype=Float32, requires_grad=True),


In [12]:
param_dict['layers.0.weight'].data

Parameter (name=layers.0.weight, shape=(64, 3, 3, 3), dtype=Float32, requires_grad=True)

In [13]:
param_dict['classifier.0.bias'].data

Parameter (name=classifier.0.bias, shape=(4096,), dtype=Float32, requires_grad=True)

In [14]:
load_param_into_net(network, param_dict)

[]

In [15]:
network.get_parameters

<bound method Cell.get_parameters of Vgg<
  (layers): SequentialCell<
    (0): Conv2d<input_channels=3, output_channels=64, kernel_size=(3, 3),stride=(1, 1),  pad_mode=pad, padding=1, dilation=(1, 1), group=1, has_bias=Falseweight_init=ones, bias_init=zeros, format=NCHW>
    (1): ReLU<>
    (2): Conv2d<input_channels=64, output_channels=64, kernel_size=(3, 3),stride=(1, 1),  pad_mode=pad, padding=1, dilation=(1, 1), group=1, has_bias=Falseweight_init=ones, bias_init=zeros, format=NCHW>
    (3): ReLU<>
    (4): MaxPool2d<kernel_size=2, stride=2, pad_mode=VALID>
    (5): Conv2d<input_channels=64, output_channels=128, kernel_size=(3, 3),stride=(1, 1),  pad_mode=pad, padding=1, dilation=(1, 1), group=1, has_bias=Falseweight_init=ones, bias_init=zeros, format=NCHW>
    (6): ReLU<>
    (7): Conv2d<input_channels=128, output_channels=128, kernel_size=(3, 3),stride=(1, 1),  pad_mode=pad, padding=1, dilation=(1, 1), group=1, has_bias=Falseweight_init=ones, bias_init=zeros, format=NCHW>
    (8):

In [16]:
# wzl
mean = [0.485 , 0.456, 0.406]
std = [0.229 , 0.224, 0.225]

def get_feature_map(tensor):
    model_cfg.logger.important_info('start get feature map')
    output = network(tensor)  
    return output

# # ccy
# def get_feature_map(img_root,img_prefix,img_id):
#     model_cfg.logger.important_info('start get feature map')
#     img_path = os.path.join(img_root,img_prefix+"_"+str(img_id).zfill(12)+".jpg")
#     print("img_path:",img_path)
    
#     dataset = ds.CocoDataset(dataset_dir=img_path,annotation_file=,task='Detection')
#     output = network(Tensor(dataset,mstype.float32))
# #     print(output)

# import mindspore.dataset as ds
# # ccy
# def get_feature_map(img_root,img_prefix,img_id):
#     model_cfg.logger.important_info('start get feature map')
#     img_path = os.path.join(img_root,img_prefix+"_"+str(img_id).zfill(12)+".jpg")
#     graph_dataset = ds.GraphData(dataset_file=img_path, num_parallel_workers=2)
#     nodes = graph_dataset.get_all_nodes(node_type=1)
#     features = graph_dataset.get_node_feature(node_list=nodes, feature_types=[1])
#     print(features)
#     output = network(Tensor(graph_dataset,mstype.float32))

In [25]:
def img2tensor(img_root,img_prefix,img_id):
        img_path = os.path.join(img_root,img_prefix+"_"+str(img_id).zfill(12)+".jpg")
        print("img_path:",img_path)
        img = Image.open(img_path).convert('RGB')
        transform = edict({
            "Decode": vision.Decode(),
            "Resize": vision.Resize((512, 512)),
            "CenterCrop": vision.CenterCrop(448),
            "Normalize": vision.Normalize(mean=mean, std=std),
            "HWC2CHW": vision.HWC2CHW(),
            "ToTensor":vision.ToTensor()
        })
        # img = transform.Decode(img)
        img = transform.Resize(img)
        img = transform.CenterCrop(img)
        img = transform.ToTensor(img)
        print("totensor:",img)
    #     img = transform.Normalize(img) # CHW
    #     print("normalized:",img)
    #     img = transform.HWC2CHW(img)
        img = [img]
        tensor = Tensor(img,mstype.float32)
        return tensor

In [26]:
img = img2tensor(model_cfg.image.train,"COCO_train2014",9)

img_path: ./data/images/train/COCO_train2014_000000000009.jpg
totensor: [[[0.         0.00392157 0.         ... 0.05098039 0.04705882 0.04705882]
  [0.         0.00784314 0.00392157 ... 0.04313726 0.04705882 0.05490196]
  [0.         0.         0.         ... 0.04313726 0.05490196 0.05490196]
  ...
  [0.827451   0.81960785 0.80784315 ... 0.03137255 0.01176471 0.04705882]
  [0.81960785 0.8156863  0.8        ... 0.01568628 0.05098039 0.14901961]
  [0.827451   0.80784315 0.79607844 ... 0.09411765 0.1764706  0.20392157]]

 [[0.13333334 0.13725491 0.13725491 ... 0.         0.         0.        ]
  [0.12941177 0.13725491 0.13333334 ... 0.         0.         0.        ]
  [0.12941177 0.14117648 0.14117648 ... 0.00392157 0.00392157 0.        ]
  ...
  [0.7058824  0.69411767 0.68235296 ... 0.20392157 0.19215687 0.20392157]
  [0.69803923 0.6901961  0.6745098  ... 0.1764706  0.19215687 0.27058825]
  [0.7058824  0.6862745  0.67058825 ... 0.22745098 0.29411766 0.30588236]]

 [[0.5176471  0.5137255 

In [27]:
img.shape

(1, 3, 448, 448)

In [28]:
out = get_feature_map(img)

2021-07-13 03:18:38,221:INFO:
**********************************************************************
**********************************************************************
**
**
**        start get feature map
**
**
**********************************************************************
**********************************************************************



In [29]:
out[0][0]

Tensor(shape=[14, 14], dtype=Float32, value=
[[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00 ...  1.70898438e+00,  9.82910156e-01,  0.00000000e+00],
 [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00 ...  1.89160156e+00,  1.40136719e+00,  2.18505859e-01],
 [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00 ...  1.78710938e+00,  2.03906250e+00,  1.13281250e+00],
 ...
 [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00 ...  1.93554688e+00,  1.36132812e+00,  0.00000000e+00],
 [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00 ...  1.32617188e+00,  1.20214844e+00,  0.00000000e+00],
 [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00 ...  0.00000000e+00,  0.00000000e+00,  0.00000000e+00]])

In [30]:
out.shape

(1, 512, 14, 14)